In [157]:
import torch as tr 
import numpy as np
from torch.autograd import Variable
from torch import optim
from sklearn.utils.extmath import fast_logdet
from sklearn.datasets import make_sparse_spd_matrix

In [2]:
random_state=0

In [49]:
A = make_sparse_spd_matrix(dim=15, alpha=0.7, random_state=random_state)

In [65]:
T_true = A[5:,5:]
K_true = A[10:,5:,]
H_true = A[0:5,0:5]
T_true.shape, K_true.shape, H_true.shape

((10, 10), (5, 10), (5, 5))

In [66]:
per_cov = K_true*0.05
T_obs = T_true - per_cov.T.dot(H_true).dot(per_cov)

In [67]:
samples = np.random.multivariate_normal(np.zeros(10), np.linalg.inv(T_obs), 50)

In [68]:
S = Variable(tr.from_numpy(samples.T.dot(samples)/50))


In [88]:
_lambda=100
_mu=10

In [133]:
# def loss_func(S, T, K, H, _lambda, _mu):
#     loss = 0
#     Hinv = H.inverse().double()
#     Hinv.type(tr.DoubleTensor)
#     T = T.double()
#     H = H.double()
#     R = T - tr.matmul(tr.matmul(tr.t(K).double(), Hinv),K.double())
#     loss += - R.logdet() + tr.trace(tr.matmul(S,R))
#     loss += _lambda*tr.norm(T, 1)
#     loss += _mu*tr.norm(H, 1)
#     return loss

def nuclear_norm(A):
    _, S_, V = tr.svd(A)
    return tr.sum(S)

def loss_func(S, T, L, _lambda, _mu):
    loss = 0
    T = T.double()
    L = L.double()
    LLT = tr.matmul(L, tr.t(L))
    R = T - LLT
    loss += - R.logdet() + tr.trace(tr.matmul(S,R))
    loss += _lambda*tr.norm(T, 1)
    loss += _mu*nuclear_norm(LLT)
    return loss

In [130]:
tr.distributions.constraints.positive_definite??

In [169]:
K = Variable(tr.from_numpy(per_cov), )
H = Variable(tr.randn(5,5), requires_grad=True)
T = Variable(tr.randn(10,10), requires_grad=True)
# for epoch in range(5000):

#     # Calcola funzione costo
#     loss = loss_func(S, T, K, H, _lambda, _mu)

#     # Esegui back-propagation
#     loss.backward()

#     # Aggiorna le variabili
#     H.data = H.data - 0.0005*H.grad.data
#     T.data = T.data - 0.0005*T.grad.data
#     # Resetta il gradiente
#     H.grad.data.zero_()
#     T.grad.data.zero_()


L = Variable(tr.from_numpy(make_sparse_spd_matrix(dim=10, alpha=0.5)))
L = Variable(tr.potrf(L, upper=False), requires_grad=True)
optimizer = optim.SGD([K, L], lr = 0.01, momentum=0.9)

for epoch in range(1000):
    optimizer.zero_grad()
    #output = model(input)
    loss = loss_func(S, T, L, _lambda, _mu)
    loss.backward()
    optimizer.step()


In [170]:
Hn = H.detach().numpy()
Hn[np.where(np.abs(Hn)<1e-1)] =0
Hn

array([[-0.11823748,  0.45512185,  1.122299  ,  0.1223267 , -0.33746904],
       [ 1.1438526 ,  1.9129025 , -0.14669214,  1.2399874 ,  0.7413741 ],
       [ 1.2472075 , -0.94150907, -0.31507027,  0.1168386 , -0.799896  ],
       [-0.8766765 , -0.7142173 , -1.1763316 , -0.86082906, -0.24611288],
       [-0.793239  ,  0.        , -0.89728177, -0.53665185, -0.2519287 ]],
      dtype=float32)

In [171]:
H_true

array([[ 1.22170946,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  2.07826367,  0.        , -0.15206798,  0.74181422],
       [ 0.        ,  0.        ,  1.        ,  0.        , -0.85945506],
       [ 0.        , -0.15206798,  0.        ,  2.22706349,  0.        ],
       [ 0.        ,  0.74181422, -0.85945506,  0.        ,  2.4664463 ]])

In [172]:
T

tensor([[ 0.0542,  1.2193, -0.2151,  0.2167,  0.1701, -1.5728,  0.8041, -0.0143,
         -1.0392, -0.4758],
        [ 0.0041,  0.5052,  1.1083, -0.3272,  1.1302,  0.0276, -1.1795,  0.5830,
          0.7042, -2.7583],
        [-0.7300,  0.7370, -0.1884, -2.9756,  1.1464,  0.0525,  0.5401, -0.4302,
         -0.0799, -0.5069],
        [-0.2369, -2.3961,  0.3244, -0.7965, -1.1581,  0.6168,  0.4607, -1.9948,
          0.7394,  0.7073],
        [ 0.9093, -0.0165, -0.0173, -1.2472,  1.1439, -0.6034, -0.8090,  1.1235,
          0.4526,  0.3039],
        [-0.4709, -2.2538,  1.4028,  0.5417, -1.0275, -0.4153,  0.0823, -1.2659,
          0.5269, -0.5990],
        [-0.6747,  1.3681, -0.1050,  0.6798, -1.6466, -0.3074, -3.0788,  0.1187,
          1.3657, -1.9998],
        [-0.7302, -0.9029,  0.5320,  1.1489,  1.0888, -1.7966, -2.8343, -1.5284,
          0.8091, -0.5398],
        [ 0.9578, -1.2211, -0.1113,  0.4318, -0.4024, -1.0155,  1.0897, -1.8786,
          0.1775,  1.0765],
        [ 0.4343,  

In [173]:
T_true

array([[ 1.        ,  0.        , -0.16673795,  0.        ,  0.        ,
        -0.88199611,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  2.04044801,  0.        , -0.63593324,  0.        ,
         0.        , -0.32538408,  0.        ,  0.27733968, -0.72812233],
       [-0.16673795,  0.        ,  1.32129891,  0.        ,  0.        ,
         0.14706222,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.63593324,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.88199611,  0.        ,  0.14706222,  0.        ,  0.        ,
         2.55713568,  0.44253476, -0.8012042 ,  0.        ,  0.        ],
       [ 0.        , -0.32538408,  0.        ,  0.        ,  0.        ,
         0.44253476,  1.30507622, -0.55233705

In [174]:
Ltrue

array([[ 2.37597182e-03,  0.00000000e+00, -3.96164667e-04,
         0.00000000e+00,  0.00000000e+00, -6.88855909e-03,
        -1.19212558e-03,  2.15832992e-03,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  5.19474519e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -2.64582991e-03,
        -6.39191241e-03,  3.30231659e-03,  5.80784117e-03,
        -5.09312291e-03],
       [-3.96164667e-04,  0.00000000e+00,  6.60556839e-05,
         0.00000000e+00,  0.00000000e+00,  1.14858421e-03,
         1.98772573e-04, -3.59875503e-04,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.

In [175]:
Ltrue - tr.matmul(L, tr.t(L)).detach().numpy()

array([[-2.18986124e+240,  6.30193522e+241,  1.24654316e+240,
         6.06528165e+241, -2.20068123e+241, -7.35571211e+239,
         7.93862973e+241,  6.21422438e+241,  2.50720354e+241,
         5.06630061e+241],
       [ 6.30193522e+241, -1.81355726e+243, -3.58727489e+241,
        -1.74545361e+243,  6.33307276e+242,  2.11681089e+241,
        -2.28456166e+243, -1.78831603e+243, -7.21517603e+242,
        -1.45796901e+243],
       [ 1.24654316e+240, -3.58727489e+241, -7.09574568e+239,
        -3.45256366e+241,  1.25270226e+241,  4.18712037e+239,
        -4.51893681e+241, -3.53734691e+241, -1.42718514e+241,
        -2.88390983e+241],
       [ 6.06528165e+241, -1.74545361e+243, -3.45256366e+241,
        -1.67990742e+243,  6.09524990e+242,  2.03731930e+241,
        -2.19877061e+243, -1.72116025e+243, -6.94422796e+242,
        -1.40321860e+243],
       [-2.20068123e+241,  6.33307276e+242,  1.25270226e+241,
         6.09524990e+242, -2.21155469e+242, -7.39205631e+240,
         7.97785410e+242

In [ ]:
from itertools import product
summ = np.zeros((3,3))

for i,j in product(range(3), range(3)):
    I = np.zeros((3,3))
    I[i,j] = 1
    summ[i,j] = Kn.dot(Rn - Tn + Kn.T.dot(Hninv).dot(Kn)).dot(Kn.T).dot((-Hninv).dot(I).dot(Hninv))

In [ ]:
summ

In [ ]:
Kn = K.numpy()
Hn = H.detach().numpy()
Rn = R.numpy()
Tn = T.numpy()

In [ ]:
np.linalg.norm(Rn - Tn + Kn.T.dot(np.linalg.inv(Hn)).dot(Kn))**2

In [ ]:
Hninv = np.linalg.inv(Hn)
I = np.zeros((3,3))
I[1,1] = 1
Kn.dot(Rn - Tn + Kn.T.dot(Hninv).dot(Kn)).dot(Kn.T).dot((-Hninv.T).dot(I).dot(Hninv))

In [ ]:
Hninv = np.linalg.inv(Hn)
(-Hninv).dot(np.ones((3,3))).dot(Hninv).dot(Kn.dot(Rn - Tn + Kn.T.dot(Hninv).dot(Kn)).dot(Kn.T))

In [ ]:
Hninv